In [ ]:
import numpy as np
import random
from Contenedor import RandomPoblation,Bin
from BPGen import PROBLEM3,PROBLEM4,PROBLEM5,PROBLEM2,PROBLEM1
from PackingH import DBLF2,DBLF
import time

In [ ]:
def CalculateFi(pobl:list,boxes:list[list[int]],cont:list[int])->list[list[float,list[int]]]:
    ffs=[]
    for ff in pobl:
        bin=Bin(cont,len(boxes))
        DBLF2(bin,ff,boxes)
        fi = 1-bin.getLoadVol()/np.prod(cont)
        ffs.append([fi,ff])
    return ffs
def SingleFi(firefly:list[float,list[int]],boxes,cont):
    bin=Bin(cont,len(boxes))
    DBLF2(bin,firefly[1],boxes)
    fi = 1-bin.getLoadVol()/np.prod(cont)
    firefly[0] = fi

In [ ]:
class Problem:
    def __init__(self, problem:list, id:int, algorithm:int):
        self.problem = problem
        self.id =id
        self.algorithm  = algorithm

In [ ]:
def DFFA(gamma:float,boxes:list[list[int]],cont:list[int],poblacion:int,Maxitr:int):
    def Hamming(arr1:list[int], arr2:list[int])->float:
        count =0
        n = len(arr1)
        for i in np.arange(n):
            if arr1[i] != arr2[i]:
                count +=1
        return count
    def Intensity(f1:list[float,list[int]],f2:list[float,list[int]],gamma:float):
        dist = Hamming(f1[1],f2[1])
        return f1[0]/(1+gamma*(dist*dist))
    def alphaStep(alpha:float,fi:list[int])->None:
        n = len(fi)
        aux = fi.copy()
        visited = []
        for i in np.arange(n):
            prev = fi[i]
            num = round(fi[i]+alpha*(random.uniform(0, 1)-0.5))
            if 0<num and num<=n:
                if num in visited:
                    continue
                visited.append(num)
                fi[i]=num
                for j in np.arange(n):
                    if fi[i]==fi[j] and i!=j:
                        fi[j]=prev
    
    def Atractiveness(f1:list[int],f2:list[int],gamma:float)->list[int]:
        n = len(f1)
        res = [None for i in np.arange(n) ]
        posible = [i+1 for i in np.arange(n)]
        used = []
        for i in np.arange(n):
            if f1[i] == f2[i]:
                res[i] = f1[i]
                used.append(f1[i])
        betta = 1/(1+gamma*(Hamming(f1,f2)**2))
        for i in np.arange(n):
            if res[i] is None:
                pr = random.random()
                if pr < betta:
                    if f2[i] in res:
                        continue
                    res[i]=f2[i]
                    used.append(res[i])
                else:
                    if f1[i] in res:
                        continue
                    res[i]=f1[i]
                    used.append(res[i])
        use = set(used)
        pos = set(posible)
        posib = list(pos-use)
        if len(posib)==0:
            return res
        elif len(posib)==1:
            for i in np.arange(n):
                if res[i] is None:
                    res[i]=posib[0]
        elif len(posib)>1:
            for i in np.arange(n):
                if res[i] is None:
                        min=np.infty
                        indx =0
                        value=0
                        for j,num in enumerate(posib):
                            localdist = np.abs(num-f2[i])
                            if localdist <= min:
                                min = localdist
                                indx=j
                                value=num
                        res[i]=value
                        posib.pop(indx)
        return res        
    bestInd =[[],[],[],[],[],[],[]]
    poblacion = RandomPoblation(N=poblacion,BoxSeq=boxes,Heuristic=False)
    ffs=CalculateFi(poblacion,boxes,cont)
    n = len(boxes)
    for _ in np.arange(Maxitr): 
        alph = n-((_+1)/Maxitr)*(n-1)
        gamma = (2.5)*( 1-(_+1)/Maxitr)*np.log(0.1/2.5)
        for i in np.arange(len(ffs)-1):
            for j in np.arange(i+1,len(ffs)):
                if Intensity(ffs[i],ffs[j],gamma) > Intensity(ffs[j],ffs[i],gamma):
                    axu = ffs[i].copy()
                    ffs[i][1] = Atractiveness(ffs[i][1],ffs[j][1],gamma)
                    while axu[1] == ffs[i][1]:
                        ffs[i][1] = Atractiveness(ffs[i][1],ffs[j][1],gamma)
                    alphaStep(alph,ffs[i][1])
                    SingleFi(ffs[i],boxes,cont)
        ffs.sort(key=lambda x:x[0])
        for i in np.arange(len(bestInd)):
           bestInd[i].append(ffs[i])
        alphaStep(alph,ffs[0][1])
        SingleFi(ffs[0],boxes,cont)
    return bestInd

In [ ]:
P4A1 = Problem(PROBLEM4,2,1)
P4A2 = Problem(PROBLEM4,2,2)
contenedor = P4A2.problem[P4A1.algorithm-1][0]
datos = P4A2.problem[P4A1.algorithm-1][1]
start_time = time.time()
bestSolutions = DFFA(gamma=1,boxes=datos,cont=contenedor,poblacion=50,Maxitr=500)
print("--- %s minutes ---" % (time.time()/60 - start_time/60))

In [ ]:
bin = Bin(contenedor,len(datos))
DBLF2(bin,bestSolutions[0][1][1],datos)
print(bin.getN(),len(datos))
bin.getLoadVol()/np.prod(contenedor)

In [ ]:
import matplotlib.pyplot as plt
problemas = P4A2
n = len(bestSolutions[1])
plt.figure(figsize=(15, 10))

plt.title("Problema"+str(problemas.id)+"  | DBLF2 | Alg."+str(problemas.algorithm)+"|No. Cajas: "+str(len(datos))+"|Contenedor: ("+str(contenedor[0])+","+str(contenedor[1])+","+str(contenedor[2])+")") 
plt.xlabel("No. Generaciones")
plt.ylabel("Aptitud")
plt.ylim(0,1)

for j in range(len(bestSolutions)):
    bestSol = [ bestSolutions[j][i][0] for i in range(n)]
    plt.plot( [i for i in range(n)],bestSol,label="Best"+str(j))
plt.legend()
plt.show()